In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split as tts

df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head(10)

In [57]:
# Checking for null values
df.isnull().sum()

In [58]:
# Dropping all the NA values from the dataset
df.dropna(inplace = True)

In [59]:
# Checking for empty string
blank = []

for index, review, label in df.itertuples():
    if type(review) == str:
        if review.isspace() == True:
            blank.append(index)
            
print(blank)

### It means we do not have empty strings in the dataset.

In [60]:
# Checking for duplicates
print(df.drop_duplicates(inplace = True))

In [61]:
df.shape

In [62]:
df['sentiment'].value_counts()

In [63]:
def clean_text1(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

cleaned1=lambda x:clean_text1(x)

In [64]:
df['review']=pd.DataFrame(df.review.apply(cleaned1))

In [65]:
df.head(10)

In [66]:
X = df['review']
y = df['sentiment']

In [67]:
# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned2=lambda x:clean_text2(x)

In [68]:
df['review']=pd.DataFrame(df.review.apply(cleaned2))
df.head(10)


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [70]:
# Building a pipeline

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

p_svc = Pipeline([('tfidf', TfidfVectorizer()),
                  ('linear', LinearSVC())])

In [71]:
#Train model by training data
p_svc.fit(X_train, y_train)


In [72]:
#make predictions using the model
predict_svc = p_svc.predict(X_test)

In [73]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, predict_svc))

In [74]:
print(metrics.classification_report(y_test, predict_svc))